# Collective progress

**Example notebook for creating anonymised, collective information on progress**

* Before running this notebook, you need to prepare the data you want to assess. To do so, please use the notebook - ""
* For testing, some example data is available in the folder "proc_data"
* Enter the name of the file that you wish to use in the first cell, after that you can run the full notebook with minimal changes. 

In [81]:
# import modules

import pandas as pd
import numpy as np
import seaborn
import matplotlib
from gst_tools.make_plots import *
import gst_tools.gst_utils as utils

import re
import os

In [82]:
# Choose what you would like to plot

#TODO - recommend a file name format to include the variable and the source. 
data_file_name = 'UN-population-data-2017.csv'
years_of_interest = ['1990', '2000', '2015']

# read the data from file and extract the key information
data = pd.read_csv('proc_data/' + data_file_name)
variable = data['variable'].unique()[0]
unit = data['unit'].unique()[0]
#years = gst_tools.get_years(data) #TODO

data_years = utils.set_countries_as_index(data)
data_years = data_years.dropna(axis=1, how='any')

#TODO - add some checks / validation

# remove comment below to display the data
#data_years

In [83]:
# Plot 1 - make a histogram

var = variable

def make_simple_histogram(df, variable, units):

    # set a style
    sns.set(style="darkgrid") 
    
    # get some basic info about the data to use for making style and annotating plot
    maximum = max(df)
    minimum = min(df)
    mean = np.mean(df)
    median = np.median(df)
    npts = len(df) 
    
    # set up the figure
    fig, axs = plt.subplots()

    # make histogram
    sns.distplot(df, kde=False,
                 bins='fd',
                 rug=False, 
                 color='mediumseagreen',
                 rug_kws={"color": "rebeccapurple", "alpha": 0.5, "linewidth": 0.4, "height": 0.03})

    
    # Dynamically set x axis range
    if minimum < 0:
        #make it symmetric
        xmin, xmax = axs.get_xlim()
        if np.absolute(xmax) > np.absolute(xmin):
            plt.xlim(-xmax, xmax)
        else:
            plt.xlim(xmin, -xmin)

    # Mathematically slightly nicer, but not ideal for this plot.
    # axs.set_xscale('log')            
                
    # Annotate the plot
    axs.annotate("max = {:.2f}".format(maximum),
                xy=(.6, 0.95), xycoords=axs.transAxes,
                fontsize=8, color='black')
    axs.annotate("min = {:.2f}".format(minimum),
                xy=(.6, 0.90), xycoords=axs.transAxes,
                fontsize=8, color='black')
    axs.annotate("mean = {:.2f}".format(mean),
                xy=(.6, 0.85), xycoords=axs.transAxes,
                fontsize=8, color='black')
    axs.annotate("median = {:.2f}".format(median),
                xy=(.6, 0.80), xycoords=axs.transAxes,
                fontsize=8, color='black')
    # TODO - make a box around this somehow?
    
    # label axes and add title
    axs.set_xlabel((var + ' (' + unit + ')'))
    axs.set_ylabel('Number of countries')
    axs.set_title((var + ' in ' + df.name), fontweight='bold')
    
    
    # save to file
    filepath = os.path.join('output', 'plots')
    fname = ('basic_histogram-' + var + '.pdf')
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    filename = os.path.join(filepath, fname)
    plt.savefig(filename, format='pdf')
    plt.close()


# run above function
# TODO - move function to background .py file
make_simple_histogram(data_years[years_of_interest[0]], variable, unit)


SyntaxError: unexpected EOF while parsing (<ipython-input-84-f02dd8c13a69>, line 10)

'ThousandPers'